In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import modules.eda as Detective
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

Đọc dữ liệu từ project 1 và sắp xếp lại các column

In [3]:
reviews = pd.read_csv("./data/normalize_reviews.csv").fillna("")
reviews = reviews[['raw_comment', 'normalize_comment', 'emoji', 'label']]

reviews.head()

,raw_comment,normalize_comment,emoji,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,😢,0


Cho đến hiện tại, chúng ta đã cùng nhau phá tan tành dataset này, điều mà tôi muốn làm tiếp theo là sẽ biễu diễn các từ lên một không gian 2 chiều hoặc thậm chí là 3 chiều.

Điều này theo tôi là rất hữu ích, vì giống như các unsupervised model, nhiều khi chúng ta cũng cần biết cách mà các observe phân phối như thế nào.

Nhưng trước tiên, ta cần một số thiết lập **ban đầu** như sau:
  * Tôi cần download vài file cấu hình sẵn:
    ```shell
    pip install fasttext
    ```
  * Thực chất, hai gói mà ta vừa download trên là chứa tập các từ tiếng anh và tiếng việt đã được vectorization thành các vector 300 dimentions, bạn đừng lo lắng về chất lượng của hai tập words embedding này bởi vì cha đẻ của nó là Facebook.
  * Thời gian download hai gói này cực kì lâu, khoảng 4GB cho một gói - và về giải nén ra mỗi file sẽ khoảng 7.3GB, tôi khuyến khích bạn đừng nên download theo cách trên mà nên lên trang chủ của họ và download trực tiếp bằng Internet download manager hoặc Xtreme download manager sẽ nhanh hơn, khoảng 15 phút cho một gói - đây là trang chủ của họ:
    [https://fasttext.cc/docs/en/crawl-vectors.html](https://fasttext.cc/docs/en/crawl-vectors.html)

In [5]:
import fasttext.util
import fasttext

fasttext.util.download_model('en', if_exists='ignore') # english
fasttext.util.download_model('vi', if_exists='ignore') # vietnamese 

'cc.vi.300.bin'

<span style="color: red; font-weight: 900; font-size: 30px">Lưu ý:</span> 
* Bạn chỉ nên sài dòng này nếu máy tính của bạn là từ 16GB RAM - Core I7 gen 7 dòng xung cao trở lên. Và nên dùng hệ điều hành Linux vì nó có RAM ảo hỗ trợ bạn để máy bạn chỉ lag thôi chứ không đứng.
  ```python
  ft = fasttext.load_model('cc.vi.300.bin')
  ```

* Còn đây là tôi đã giảm chiều từ một bộ data 7.2GB xuống còn 100 chiều, bạn có thể dùng dòng này, bạn có thể giảm nhiều hơn nếu muốn. Và tốt nhất là bạn cần tắt tất cả mọi chương trình nên đang chạy nếu bạn không muốn nghe máy tính của bạn "hát ca điên cuồng".
  ```python
  fasttext.util.reduce_model(ft, 100)
  ft.save_model('cc.vi.100.bin')
  ```

Bây giờ chúng ta sẽ load lại tập embed word 100 dims mà ta vừa tạo, nó sẽ ăn máy của bạn khoảng 3GB RAM.

In [9]:
ft = fasttext.load_model('cc.vi.100.bin')

Dưới đây là vector 100 dims của từ "xin chào".

Thực chất fasttext là một công cụ hỗ trợ cực kì tốt mà tôi biết đến nó thông qua một tài tweet tren Twitter. Hãy tin tưởng mà sử dụng nó, nó sẽ giúp cho cái bài toán NLP của bạn "đỡ khổ" hơn nhiều.

In [10]:
ft.get_word_vector('xin chào')

array([-0.005858  ,  0.00591698,  0.05544396,  0.01845348,  0.03497462,
       -0.01731673, -0.0186325 , -0.05789406, -0.01456467,  0.05405328,
        0.0162955 ,  0.02371605, -0.02689386,  0.0168477 , -0.02608746,
       -0.00542725, -0.01952248,  0.02309453,  0.00077463, -0.01623438,
       -0.02708885, -0.02166499,  0.01338364, -0.00763521, -0.01389443,
       -0.00743966, -0.04979044, -0.02639596,  0.01744116, -0.02117856,
        0.00118052,  0.00367191, -0.00203112,  0.01383345,  0.01485165,
       -0.00397428,  0.02199381,  0.02537224, -0.00114648, -0.01440865,
       -0.01821862,  0.01354136,  0.00441786, -0.01989009,  0.00784927,
       -0.00614514,  0.01168528,  0.02575686,  0.01241738, -0.00843297,
        0.02774723,  0.00453759,  0.00838433,  0.01327751, -0.01783348,
        0.01831022, -0.01594335, -0.00435727,  0.00035578, -0.07555512,
        0.01310887, -0.04912862,  0.01182956,  0.03159093,  0.04497012,
        0.02143607,  0.00049193, -0.00196764,  0.00246742,  0.01